In [0]:
dbutils.widgets.text("catalog", "etl_demo")  
catalog        = dbutils.widgets.get("catalog").strip()
spark.sql(f"USE CATALOG {catalog}")
print("CATALOG:", catalog)


In [0]:
%sql


CREATE SCHEMA IF NOT EXISTS silver_staging;

CREATE OR REPLACE TABLE silver_staging.customer_snapshot_stg
USING DELTA
AS
WITH base AS (
  SELECT
    customer_id,
    name,
    city,
    segment,
    snapshot_date,
    _ingest_time,
    _file_path
  FROM bronze.customer_bronze
  WHERE customer_id IS NOT NULL
    AND snapshot_date IS NOT NULL
),
latest_snapshot AS (
  SELECT MAX(snapshot_date) AS snapshot_date
  FROM base
),
filtered AS (
  SELECT b.*
  FROM base b
  JOIN latest_snapshot l
    ON b.snapshot_date = l.snapshot_date
),
dedup AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY customer_id, snapshot_date
           ORDER BY _ingest_time DESC
         ) AS rn
  FROM filtered
)
SELECT
  customer_id,
  name,
  city,
  segment,

  -- ✅ row_hash computed here (change detection)
  sha2(
    concat_ws('||',
      coalesce(CAST(name AS STRING), ''),
      coalesce(CAST(city AS STRING), ''),
      coalesce(CAST(segment AS STRING), '')
    ),
    256
  ) AS row_hash,

  snapshot_date,
  _ingest_time,
  _file_path
FROM dedup
WHERE rn = 1;


In [0]:
print("test")